#### This notebook has been tested with SageMaker Studio Notebooks, with Data science 3.0 and Python 3 environment

In [122]:
!pip install -q langchain==0.0.264

In [123]:
import langchain
print(langchain.__version__)

0.0.264


In [124]:
import boto3
aws_region = boto3.Session().region_name
print(aws_region)
llm_endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-7b-f"
print(endpoint_name)

us-east-1
jumpstart-dft-meta-textgeneration-llama-2-7b-f


In [125]:
from langchain import PromptTemplate

prompt_template = """

Please help me to find the answer, based on the retrieved information and the user input question.

user input question:

{user_query}

retrieved information:

{retrieved_information}

Please ensure the answer are relevant to retrieved information, if you could not find the answer, please wrote #IDONTKNOW exact words without any extrat word. 

"""
prompt = PromptTemplate.from_template(prompt_template)

In [126]:
from langchain.prompts import StringPromptTemplate
import pandas as pd
from sklearn.preprocessing import normalize
import numpy as np 

import psycopg2
from pgvector.psycopg2 import register_vector
import boto3 
import json 


class RetrievedPromptTemplate(StringPromptTemplate):
    embedding_endpoint_name = "jumpstart-dft-mx-tcembedding-robertafin-large-uncased-2"
    
    def format(self, **kwargs) -> str:
        user_query = kwargs.pop("user_query")
        query_embedding = self.fetch_user_query_embedding(user_query)
        retrieved_information = self.fetch_retrieved_information_from_pgvector(query_embedding)
        
        kwargs["user_query"] = user_query
        kwargs["retrieved_information"] = retrieved_information
        return prompt.format(**kwargs)
    
    
    def fetch_retrieved_information_from_pgvector(self, query_vector):

        client = boto3.client('secretsmanager')

        response = client.get_secret_value(
            SecretId='rdspg-vector-secret'
        )
        database_secrets = json.loads(response['SecretString'])
        
        dbhost = database_secrets['host']
        dbport = database_secrets['port']
        dbuser = database_secrets['username']
        dbpass = database_secrets['password']

        dbconn = psycopg2.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10)
        dbconn.set_session(autocommit=True)
        cur = dbconn.cursor()
        
        query_vector = str(query_vector.tolist())
        #print(type(query_vector))

        # Execute the query
        cur.execute("""
            SELECT id, content, pdf_file_name, page_number, pdf_file_path, content_embeddings
            FROM pdfembedding
            ORDER BY content_embeddings <=> %s limit 3;
            """,(query_vector,)
        )
        
        # Fetch the results
        content = " "
        results = cur.fetchall()
        for row in results:
            content += row[1]+ "\n"*5 
            
        cur.close()
        dbconn.close()
        #print(content)
        return content
    

    def chunk_words(self,sequence, chunk_size):
        sequence = sequence.split()
        return [' '.join(sequence[i:i+chunk_size]) for i in range(0, len(sequence), chunk_size)]

    def query_endpoint(self,payload):
        embeddings = []
        client = boto3.client("sagemaker-runtime")
        chunk_payload = self.chunk_words(payload, 400)
        for i, chunk in enumerate(chunk_payload):
            #print("Chunk ",i)
            #print("Content ",chunk)
            response = client.invoke_endpoint(
                EndpointName=self.embedding_endpoint_name,
                ContentType="application/x-text",
                Body=json.dumps(chunk),
            )    
            response = response["Body"].read().decode("utf8")
            response = json.loads(response)
            embeddings_chunk = response["embedding"]
            embeddings.append(embeddings_chunk)
        return embeddings
    
    def parse_response(self, query_response):
        """Parse response and return the embedding."""
        embeddings = np.array(self.query_endpoint(query_response))
        #avg_embeddings = np.mean(embeddings, axis=0)
        # try max pooling of embedding vector
        avg_embeddings = np.max(embeddings, axis=0)

        avg_embeddings = avg_embeddings.reshape(1, -1)
        # normalization before inner product
        avg_embeddings = normalize(avg_embeddings, axis=1, norm='l2')
        return np.squeeze(avg_embeddings)


    def fetch_user_query_embedding(self, user_query): 
        return self.parse_response(user_query)

In [127]:
prompt_template = RetrievedPromptTemplate(input_variables=["user_query"])


In [128]:
Question = """
Who is the Chairman of the Board and Chief Executive Officer
"""

In [129]:
print(prompt_template.format(user_query=Question))



Please help me to find the answer, based on the retrieved information and the user input question.

user input question:


Who is the Chairman of the Board and Chief Executive Officer


retrieved information:

 45
2022 ANNUAL REPORT
Directors and Senior Management 
The Directors of the Company during the Reporting Period and up to the date of the annual report were: 
Executive Directors
Lei Jun
Lin Bin
Liu De
Non-Executive Director
Liu Qin
Independent Non-Executive Directors
Chen Dongsheng 
Wong Shun Tak 
Tong Wai Cheung Timothy 
The Company has received from each independent non-executive Director an annual confirmation of his independence 
pursuant to Rule 3.13 of the Listing Rules and the Board considers them as independent.
Biographical Details and Other Information of the Directors
The biographical details of the Directors who held office during the Reporting Period and up to the Latest Practicable Date are as out below:
Lei Jun (雷軍) , aged 53, is an executive Director, the Foun

In [130]:
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({"inputs" : [[{"role" : "system",
        "content" : "You are an accuracy information searching AI assitance."},
        {"role" : "user", "content" : prompt}]],
        "parameters" : {**model_kwargs}})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generation"]["content"]
    

parameters = {
    "max_new_tokens": 500,
    "top_p":0.9, 
    "temperature":0.01
}

content_handler = ContentHandler()

sm_llm = SagemakerEndpoint(
    endpoint_name = llm_endpoint_name, 
    region_name = aws_region, 
    model_kwargs = parameters,
    endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
    content_handler = content_handler,
)

In [131]:
Question = """
Who is the Chairman of the Board and Chief Executive Officer
"""

In [132]:
from langchain.chains import LLMChain


llmchain = LLMChain(llm=sm_llm, prompt=prompt_template)
rag_results = llmchain.run({
    'user_query': Question,
    })

# OUTPUT
# PodConneXion

In [133]:
print(rag_results)

 Based on the retrieved information, the answer to the user's question "Who is the Chairman of the Board and Chief Executive Officer of Xiaomi Corporation?" is:

Lei Jun.

According to the information provided in the annual report, Lei Jun is the Chairman of the Board and Chief Executive Officer of Xiaomi Corporation. He is also an executive Director and the founder of the company.


In [136]:
Question = """
What is the main business of Xiaomi
"""

In [137]:
from langchain.chains import LLMChain

rag_results = llmchain.run({
    'user_query': Question,
    })
print(rag_results)

 Based on the retrieved information, the main business of Xiaomi can be summarized as:

1. Smartphones: Xiaomi is a leading smartphone manufacturer that offers a range of high-quality, affordable smartphones with innovative features and designs.
2. AIoT (Artificial Intelligence of Things): Xiaomi is also a major player in the AIoT market, offering a range of smart home devices, wearables, and other AI-enabled products that integrate with its smartphones and other devices.
3. Offline Channels: Xiaomi has a strong offline channel presence, with over 10,000 retail stores in mainland China, and it continues to optimize its offline channel business, including display layouts, product mix, and cross-selling promotions.
4. Corporate Social Responsibility: Xiaomi highly values corporate social responsibility and actively promotes sustainable development through various initiatives, including donations to charitable organizations and environmental protection efforts.

In response to the user in